In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# Stats
from scipy import stats

# Load dataset
df = pd.read_csv("/kaggle/input/ecommerceedasalesdataset/sales_data_sample.csv", encoding='ISO-8859-1')

# Quick overview
print(df.head())
print(df.info())
print(df.describe())
print(df.columns)

# Convert date column to datetime (replace 'OrderDate' with your column)
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Fill missing numeric values with 0
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(0)

# Fill missing categorical values with 'Unknown'
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

df['Month'] = df['OrderDate'].dt.month
df['Weekday'] = df['OrderDate'].dt.day_name()

# Total Revenue (Quantity * Price per unit)
df['TotalRevenue'] = df['Quantity'] * df['Price']

plt.figure(figsize=(10,5))
sns.histplot(df['TotalRevenue'], bins=50, kde=True)
plt.title("Revenue Distribution")
plt.xlabel("Revenue")
plt.ylabel("Frequency")
plt.show()

monthly_sales = df.groupby('Month')['TotalRevenue'].sum()
monthly_sales.plot(kind='bar', figsize=(10,5), color='skyblue')
plt.title("Monthly Sales Revenue")
plt.xlabel("Month")
plt.ylabel("Total Revenue")
plt.show()

top_categories = df.groupby('Category')['TotalRevenue'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=top_categories.values, y=top_categories.index, palette='viridis')
plt.title("Top 10 Product Categories by Revenue")
plt.xlabel("Total Revenue")
plt.show()

df['Revenue_zscore'] = stats.zscore(df['TotalRevenue'])

# Detect anomalies (z-score > 3 or < -3)
outliers = df[np.abs(df['Revenue_zscore']) > 3]
print(f"Number of anomalies detected: {len(outliers)}")

mean_revenue = df['TotalRevenue'].mean()
std_revenue = df['TotalRevenue'].std()
n = len(df)
conf_interval = stats.norm.interval(0.95, loc=mean_revenue, scale=std_revenue/np.sqrt(n))
print(f"95% Confidence Interval for Average Revenue: {conf_interval}")

daily_revenue = df.groupby('OrderDate')['TotalRevenue'].sum()
plt.figure(figsize=(14,6))
plt.plot(daily_revenue.index, daily_revenue.values, marker='o')
plt.title("Daily Revenue Trend")
plt.xlabel("Date")
plt.ylabel("Total Revenue")
plt.show()

weekday_sales = df.groupby('Weekday')['TotalRevenue'].sum().reindex(
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
weekday_sales.plot(kind='bar', figsize=(10,5), color='orange')
plt.title("Weekday Revenue Trend")
plt.xlabel("Weekday")
plt.ylabel("Total Revenue")
plt.show()


numeric_cols = ['Quantity','Price','TotalRevenue']
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


   ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10159               49     100.00               14  5205.27   

         ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  ...  \
0   2/24/2003 0:00  Shipped       1         2     2003  ...   
1    5/7/2003 0:00  Shipped       2         5     2003  ...   
2    7/1/2003 0:00  Shipped       3         7     2003  ...   
3   8/25/2003 0:00  Shipped       3         8     2003  ...   
4  10/10/2003 0:00  Shipped       4        10     2003  ...   

                    ADDRESSLINE1  ADDRESSLINE2           CITY STATE  \
0        897 Long Airport Avenue           NaN            NYC    NY   
1             59 rue de l'Abbaye           NaN

KeyError: 'OrderDate'

In [2]:
import pandas as pd

# Load dataset (with encoding if needed)
df = pd.read_csv("/kaggle/input/ecommerceedasalesdataset/sales_data_sample.csv", encoding='ISO-8859-1')

# Check the column names
print(df.columns)

# Convert date column to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Feature engineering
df['Month'] = df['Order Date'].dt.month
df['Weekday'] = df['Order Date'].dt.day_name()

# Total Revenue
df['TotalRevenue'] = df['Quantity Ordered'] * df['Price Each']

# Drop rows with missing or invalid dates
df = df.dropna(subset=['Order Date'])

# Convert numeric columns to numbers if needed
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors='coerce')
df['Price Each'] = pd.to_numeric(df['Price Each'], errors='coerce')

# Drop rows with NaN after conversion
df = df.dropna(subset=['Quantity Ordered', 'Price Each'])


Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')


KeyError: 'Order Date'